# Метод косинусного сходства

## О методе

Метод **косинусного сходства** (*cosine similarity*) в коллаборативной фильтрации используется для измерения сходства между пользователями или предметами на основе их характеристик или предпочтений.

Косинусное сходство измеряет **угол** между двумя **нормированными** векторами в многомерном пространстве.

- Если угол маленький (векторы направлены в одну сторону), сходство близко к 1 (*почти идентичные предпочтения*)
- Если угол большой (векторы направлены в разные стороны), сходство близко к 0 (*нет связи*)
- Если векторы полностью противоположны, сходство -1 (*полная антипатия, но такое можно встретить крайне редко, отрицательные значения можно, например, обнулять или брать по модулю, рекомендательной системе важнее выявить сходства, чем противоположные предпочтения*)

Косинус угла $ \theta $ между векторами $ \mathbf{A} $ и $ \mathbf{B} $ находится по формуле:

$$
\cos \theta = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|}
$$

где:

- $ \mathbf{A} \cdot \mathbf{B} = \sum_{i=1}^{n} A_i B_i = A_1 B_1 + A_2 B_2 + \dots + A_n B_n $ — скалярное произведение векторов,
- $ \|\mathbf{A}\| $ и $ \|\mathbf{B}\| $ — их длины (модули), вычисляемые как:

$$
\|\mathbf{A}\| = \sqrt{\sum_{i=1}^{n} A_i^2} = \sqrt{A_1^2 + A_2^2 + \dots + A_n^2}, \|\mathbf{B}\| = \sqrt{\sum_{i=1}^{n} B_i^2} = \sqrt{B_1^2 + B_2^2 + \dots + B_n^2}
$$

## Реализация

### Данные

**MIND** (***Mi**crosoft* ***N**ews* ***D**ataset*) - крупномасштабный набор данных для исследования рекомендаций новостей. Содержит около 160 тыс. новостных статей на английском языке и более 15 млн. журналов показов, созданных 1 млн. пользователей.

Будет использоваться **MIND-small** - около 50 тыс. пользователей и их журналы взаимодействия.

Данные состоят из нескольких файлов, среди которых интересующий нас - журнал взаимодействия пользователей (*behaviors.tsv*)

Файл содержит таблицу, каждая строка которой состоит из номера взаимодействия, id пользователя, даты и времени взаимодействия, показанных пользователю новостей, новостей, на которые пользователь кликнул.

In [3]:
import pandas as pd

df = pd.read_csv('../../data/Mindsmall_train/behaviors.tsv', sep='\t', header=None, names=['id', 'user_id', 'timestamp', 'shown_news', 'clicked_news'])

print(df.head(2))

   id user_id              timestamp  \
0   1  U13740  11/11/2019 9:05:58 AM   
1   2  U91836  11/12/2019 6:11:30 PM   

                                          shown_news  \
0  N55189 N42782 N34694 N45794 N18445 N63302 N104...   
1  N31739 N6072 N63045 N23979 N35656 N43353 N8129...   

                                        clicked_news  
0                                  N55689-1 N35729-0  
1  N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...  


### Идея

Для реализации метода косинусного сходства нужны две вещи: векторы и сущности, которые они представляют (в нашем случае - пользователи). В качестве вектора можно представить список новостей, на которые пользователь кликнул.

Возможных значений координат вектора всего два:

- 1 - если пользователь кликнул на новость
- 0 - если не кликнул

Вектор будет состоять из **всех** известных посещенных пользователем новостей.

Нужно создать **матрицу пользователь-новость**, которая показывает, какие новости просматривал каждый пользователь. Эта матрица будет использоваться для вычисления косинусного сходства и рекомендаций.

### Код

Столбцы **user_id** и **clicked_news** содержат строковые значения, причем столбец clicked_news может содержать сразу несколько новостей. Сначала нужно преобразовать таблицу таким образом, чтобы в каждой строке значение столбца clicked_news было одним. Затем закодировать их, чтобы представить значения в виде чисел для удобной работы с матрицами. Для этого можно использовать метод `factorize()`, который:

- Преобразует **строковые значения** в **числовые индексы**
- Возвращает два массива: **кодированные значения** и **уникальные категории**

In [4]:
import numpy as np

df = df[['user_id', 'clicked_news']]
df['clicked_news'] = df['clicked_news'].str.split()
df = df.explode('clicked_news')
users, user_idx = pd.factorize(df['user_id'])
news, news_idx = pd.factorize(df['clicked_news'])

print(users)
print(news)
len(users) == len(news)

[    0     0     1 ... 13308 13308 13308]
[   0    1    2 ... 4178 1233 1574]


True

In [5]:
user_news_matrix = np.zeros((len(user_idx), len(news_idx)), dtype=int)
print(user_news_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [6]:
user_news_matrix[users, news] = 1
# Проверка правильности:
print("Первые 5 пользователей:", user_idx[:5])
print("Первые 6 новостей:", news_idx[:6])
print("Фрагмент матрицы пользователь-новость:")
print(user_news_matrix[:5, :6])  # Фрагмент 5x6 для проверки

Первые 5 пользователей: Index(['U13740', 'U91836', 'U73700', 'U34670', 'U8125'], dtype='object')
Первые 6 новостей: Index(['N55689-1', 'N35729-0', 'N20678-0', 'N39317-0', 'N58114-0', 'N20495-0'], dtype='object')
Фрагмент матрицы пользователь-новость:
[[1 1 0 0 0 0]
 [0 0 1 1 1 1]
 [0 0 1 1 0 0]
 [0 1 0 0 0 0]
 [0 0 1 1 1 0]]


Получена матрица, строки которой - это те векторы, которые и нужно сравнивать. Теперь можно реализовать сам метод косинусного сходства - вычисление угла между векторами

In [7]:
def cosine_similarity_matrix(matrix):
    # Нормализация векторов
    norms = np.linalg.norm(matrix, axis=1, keepdims=True)
    normalized_matrix = matrix / norms

    # Получаем матрицу сходства, умножая нормализованную матрицу на транспонированную
    similarity_matrix = np.dot(normalized_matrix, normalized_matrix.T)

    return similarity_matrix

cos_sim_matrix = cosine_similarity_matrix(user_news_matrix)
print(cos_sim_matrix)

[[1.         0.16057266 0.00698857 ... 0.         0.11268723 0.01050814]
 [0.16057266 1.         0.16790507 ... 0.0371023  0.01781176 0.00664382]
 [0.00698857 0.16790507 1.         ... 0.09688768 0.         0.0231326 ]
 ...
 [0.         0.0371023  0.09688768 ... 1.         0.         0.        ]
 [0.11268723 0.01781176 0.         ... 0.         1.         0.        ]
 [0.01050814 0.00664382 0.0231326  ... 0.         0.         1.        ]]


In [12]:
print(cos_sim_matrix[:5, :5])

[[1.         0.16057266 0.00698857 0.02129589 0.01356595]
 [0.16057266 1.         0.16790507 0.05385771 0.11150278]
 [0.00698857 0.16790507 1.         0.09376145 0.14932027]
 [0.02129589 0.05385771 0.09376145 1.         0.04550158]
 [0.01356595 0.11150278 0.14932027 0.04550158 1.        ]]


In [14]:
df_matrix = pd.DataFrame(cos_sim_matrix)
df_matrix.to_csv('cos_sim_matrix.csv', index=True)

KeyboardInterrupt: 